In [1]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [2]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [3]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [4]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [5]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [6]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [7]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [8]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

[PROFILE] __getitem__ took 0.001174 seconds
I am not here
{'spec': tensor([[[4.1434e+02, 1.1000e+00],
         [5.8066e+01, 5.2471e-04],
         [6.7055e+01, 1.7872e-03],
         ...,
         [3.9730e+02, 7.6047e-04],
         [4.1434e+02, 1.0000e+00],
         [4.1652e+02, 4.1300e-04]],

        [[4.0523e+02, 1.1000e+00],
         [5.7070e+01, 1.8700e-02],
         [6.9033e+01, 2.6120e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.5306e+02, 1.1000e+00],
         [3.3905e+02, 6.2000e-02],
         [3.4356e+02, 1.0000e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[6.2331e+02, 1.1000e+00],
         [5.8065e+01, 3.3278e-03],
         [1.2106e+02, 1.7990e-03],
         ...,
         [6.0728e+02, 3.5027e-03],
         [6.0829e+02, 1.8715e-03],
         [6.2331e+02, 1.0000e+00]],

        [[4.231

In [9]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])


In [10]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [11]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [12]:
tmp_original['precursor_mz']

tensor([414.3370, 405.2272, 353.0600, 818.5700, 369.1810, 551.1031, 539.1200,
        627.0900, 331.0812, 623.3120, 423.1737, 295.0600])

In [13]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+'],)

In [14]:
tmp_original['mol_freq']

tensor([ 61.,   5.,  15.,  10.,   2.,  17.,  93.,   2.,   4.,  68.,   3., 139.])

In [15]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [16]:
sum(tmp_original['labels'])

tensor(12)

In [17]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1536, 1792, 2048, 2304, 2560, 2816])

In [18]:
len(tmp_original['labels'])

3072

In [19]:
tmp_original['batch_ptr']

tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])

In [20]:
sum(tmp_original['batch_ptr'])

tensor(3072)

In [21]:
len(tmp_original['candidates'])

3072

In [22]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [23]:
len(tmp_original['candidates_smiles'])

3072

# MSnRetrieval

In [24]:
from massspecgym.featurize import SpectrumFeaturizer

In [25]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [26]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [27]:
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.json"

In [29]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/min_sample_trees.mgf"


In [30]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 105
MSnRetrievalDataset length: 105


In [31]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [32]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 82
Val dataset size: 6


In [33]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

[PROFILE] __getitem__ took 0.002131 seconds
[PROFILE] __getitem__ took 0.001138 seconds
[PROFILE] __getitem__ took 0.001005 seconds
{'spec': DataBatch(x=[128, 1039], edge_index=[2, 232], batch=[128], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([540.2440, 371.1853, 253.0495, 482.2901, 230.0812, 433.2221, 393.1333,
        401.1959, 423.2014, 435.1286, 305.1747, 325.1434]), 'adduct': ['[M+NH4]+', '[M+H]+', '[M+H-H2O]+', '[M+NH4]+', '[M+H]+', '[M+H]+', '[M+H-H2O]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+'], 'identifier': ['0000433_0000000', '0000205_0000000', '0000236_0000000', '0000625_0000000', '0000468_0000000', '0000151_0000000', '0000126_0000000', '0000272_0000000', '0000319_0000000', '0000423_0000000', '0000560_0000000', '0000

In [34]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256],
       dtype=torch.int32)


In [35]:
def print_key_types(obj, obj_name):
    print(f"Data types in {obj_name}:")
    for key, value in obj.items():
        print(f"  {key}: {type(value)}")

# Print data types for both objects
print_key_types(tmp_original, "tmp_original")
print_key_types(tmp_msn, "tmp_msn")

Data types in tmp_original:
  spec: <class 'torch.Tensor'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  identifier: <class 'list'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>
Data types in tmp_msn:
  spec: <class 'abc.DataBatch'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  identifier: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [36]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [37]:
tmp_msn['spec'] 

DataBatch(x=[128, 1039], edge_index=[2, 232], batch=[128], ptr=[13])

In [38]:
tmp_msn['precursor_mz']

tensor([540.2440, 371.1853, 253.0495, 482.2901, 230.0812, 433.2221, 393.1333,
        401.1959, 423.2014, 435.1286, 305.1747, 325.1434])

In [39]:
tmp_msn['mol_freq']

tensor([2., 1., 2., 2., 1., 4., 2., 2., 1., 1., 1., 1.])

In [40]:
tmp_msn['identifier']

['0000433_0000000',
 '0000205_0000000',
 '0000236_0000000',
 '0000625_0000000',
 '0000468_0000000',
 '0000151_0000000',
 '0000126_0000000',
 '0000272_0000000',
 '0000319_0000000',
 '0000423_0000000',
 '0000560_0000000',
 '0000484_0000000']

In [41]:
tmp_msn['adduct']

['[M+NH4]+',
 '[M+H]+',
 '[M+H-H2O]+',
 '[M+NH4]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H-H2O]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+']

In [42]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [43]:
tmp_msn['smiles']

['COC1=C(C=C2C(C(C(CC2=C1)CO)CO[C@H]3[C@@H]([C@H]([C@@H]([C@H](O3)CO)O)O)O)C4=CC(=C(C=C4)O)OC)O',
 'CC1C(C2=C(C=C3C(=C2O1)C(=O)C(=C(C3=O)OC)C)O)(C)CCC=C(C)C',
 'CC1=CC(=C2C(=C1)OC(=O)C3=C(C2=O)C(=CC=C3)O)O',
 'CC1=C2C(C(C(C3C24CCC(C4)(C(=O)C3)C)OC(=O)C)OC(=O)C5=CC=CC=C5)C(CC1)(C)C',
 'COC1=CC2=C(C=C1)N=C3C(=C2OC)C=CO3',
 'C[C@H]1CC2=CC(=C(C(=C2C3=C(C(=C(C=C3C[C@]1(C)O)OC)OC)OC)OC)OC)OC',
 'CC(=C)[C@H]1CC2=C(O1)C=CC3=C2O[C@H]4[C@@H](C3=O)C5=CC(=C(C=C5OC4O)OC)OC',
 'CC1CC2=CC3=C(C(=C2C4=C(C(=C(C=C4CC1C)OC)OC)OC)OC)OCO3',
 'C/C(=C/COC(=O)C)/C(=O)OC1CC(C(C(C1C(=C)C(=O)O)O)C(=C)CO)(C)C=C',
 'C1[C@H](OC2=C(C1=O)C(=C(C(=C2)O)[C@H]3[C@@H]([C@H]([C@@H]([C@H](O3)CO)O)O)O)O)C4=CC=C(C=C4)O',
 'CCCCC1=C(C=C2CC(OC(=O)C2=C1O)CC/C=C/C)O',
 'CC(=CCC1=C(C=CC(=C1)C2CC(=O)C3=C(O2)C=C(C=C3)O)O)C']

In [44]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [45]:
sum(tmp_msn['labels'])

tensor(12)

In [46]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1536, 1792, 2048, 2304, 2560, 2816])

In [47]:
len(tmp_msn['labels'])

3072

In [48]:
tmp_msn['batch_ptr']

tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256],
       dtype=torch.int32)

In [49]:
sum(tmp_msn['batch_ptr'])

tensor(3072, dtype=torch.int32)

In [50]:
len(tmp_msn['candidates'])

3072

In [51]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [52]:
tmp_msn.keys() == tmp_original.keys()

True

In [53]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [54]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

False

In [55]:
tmp_original['batch_ptr'].dtype

torch.int64

In [56]:
tmp_msn['batch_ptr'].dtype

torch.int32

In [57]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(3072, dtype=torch.int32), 3072)

In [58]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(3072), 3072)

In [59]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [60]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [61]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [62]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [63]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [64]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [65]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [66]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [67]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [68]:
len(tmp_msn['candidates_smiles'])

3072